# trainset data prior analysis
- down to to 10 meaningful features
- standardize

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv(f'../datasets/dataset_train.csv')
df_class = df['Hogwarts House']
df_class.head()

# df['Hogwarts House'] works for only 1 col

In [ ]:
df_features = df[df.columns[6:]]
df_features.head()
# removed non_num = ['Index', 'First Name', 'Last Name', 'Birthday', 'Best Hand',]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.scatterplot(
 data=df, 
 x="Astronomy",
 y="Defense Against the Dark Arts",
 hue="Hogwarts House",
 legend='auto'
)

   Only the meaningful variables should be included.
    The independent variables should be independent of each other. That is, the model should have little or no multicollinearity.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.scatterplot(
 data=df, 
 x="Astronomy",
 y="Defense Against the Dark Arts",
 hue="Hogwarts House",
 legend='auto'
)

Removing

In [ ]:
sns.boxplot(data=df, x="Hogwarts House", y="Arithmancy")

In [ ]:
sns.boxplot(data=df, x="Hogwarts House", y="Care of Magical Creatures")

    Only the meaningful variables should be included.
    The independent variables should be independent of each other. 
    That is, the model should have little or no multicollinearity.

In [ ]:
excluded_features = ["Arithmancy", "Defense Against the Dark Arts", "Care of Magical Creatures"]
df_features = df_features.drop(excluded_features, axis=1)
df_features.head()


In [ ]:
features = df_features.columns.to_list()
print(features)

Standardization
```df_features```
apply along axis

In [ ]:
df_features.apply(np.mean) #axis=0


In [ ]:
def standardize(arr:np.ndarray):
    mean = np.mean(arr)
    std = np.std(arr)
    return (arr - mean) / std

df_std_features = df_features.agg(lambda course:standardize(course))
df_std_features.describe()

In [ ]:
sns.boxplot(data=df_std_features)

Remove duplicates if any
remove full na row if any

getting the 4 classes in a list

In [ ]:
df['Hogwarts House'].unique()
# houses[0]

one-vs-all :
actual class y set to 1 or 0
1= is in house, 0= is in another house

In [ ]:
houses = df['Hogwarts House'].unique()
houses[0]
# gives bool : df['Hogwarts House'] == houses[0]
y_actual = np.where(df['Hogwarts House'] == houses[0], 1, 0)
y_actual

dot product :
X.weights = output

tranpose weights : weights is a vertical array. 1600

transposed
```weights = np.ones(1 + len(features)).T```

In [ ]:
#x_train = np.array(df_std_features)
x_train = np.array(df_std_features.fillna(0))
weights = np.ones(len(features)).T
z_output = np.dot(x_train, weights)
z_output

Sigmoid function (or logistic function) to map input values from a wide range into a limited interval. 
$sigmoid function$
$$
y = g(z) = \frac{1}{1 + e^{-z}} = \frac{e^z}{1 + e^z}
$$
This formula represents the probability of observing the output y = 1 of a Bernoulli random variable. This variable is either 1 or 0 (y \in {0,1}).

In [ ]:
def sigmoid(arr:np.ndarray):
    return 1 / (1 + np.exp(-arr))

h_pred = sigmoid(z_output)
print(h_pred)

In [ ]:
print(len(z_output), len(h_pred), len(y_actual))

Latex examples

\begin{align} \mathbf{a} \cdot \mathbf{b} &= \sum_{i=1}^n a_i b_i \\ &= a_x b_x + a_y b_y + a_z b_z. \end{align}

\begin{align} \sigma^2 = \begin{pmatrix} \sigma_1^2 & \sigma_{12}^2 \\ \sigma_{12}^2 & \sigma_2^2 \end{pmatrix}. \end{align}

\begin{align} P(\sigma) = \left\{ \begin{array}{cl} \sigma^{-1} & \sigma > 0 \\ 0 & \sigma \le 0. \end{array} \right. \end{align}

 \begin{pmatrix} \sigma_1^2 & \sigma_{12}^2 \\ \sigma_{12}^2 & \sigma_2^2 \end{pmatrix}. 

In [ ]:
a = np.array([[1,2,0.5 ],[2,0,1],[0,np.nan,0.8,],[2.5,1.2,0.4]]) 
b = np.array([[2,3,4]]) 
print(np.dot(a, b.T))

In [ ]:
# mask array to avoi NaN
v1_m = np.ma.array(a, mask=np.isnan(a))
v2_m = np.ma.array(b, mask=np.isnan(b))
print(np.ma.dot(v1_m, v2_m.T))

In [ ]:
def loss(y_actual, h_pred):
    """ y_actual : target class. 1 in class, 0 not in class
    h_pred = signoid(x.weights)
    loss = (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    """
    m = len(h_pred)
    a = -y_actual * np.log(h_pred)
    b = (1 - y_actual) * np.log(1 - h_pred)
    return (a - b) / m

loss(y_actual, h_pred)


The weights are updated by substracting the derivative (gradient descent) times the learning rate,
loss'(theta) = 
def gradient_descent(X, h, y):
    return np.dot(X.T, (h - y)) / y.shape[0]
def update_weight_loss(weight, learning_rate, gradient):
    return weight - learning_rate * gradient

In [ ]:
v1_m = np.ma.array(x_train, mask=np.isnan(x_train))
res = (h_pred - y_actual)
v2_m = np.ma.array(res, mask=np.isnan(res))
#dot = np.ma.dot(x_train, v2_m.T)
dot = np.ma.dot(v1_m.T, v2_m)
gradient1 = dot / y_actual.shape[0]
gradient1

replace np.nan with zeros
```x_train[np.isnan(x_train)] = 0 ```

In [ ]:
dot = np.dot(x_train.T, (h_pred - y_actual))
gradient = dot / y_actual.shape[0]
gradient

In [ ]:
def gradient_descent(x_train, h_pred, y_actual):
    return np.dot(x_train.T, (h_pred - y_actual)) / y_actual.shape[0]

def update_weight_loss(weight, learning_rate, gradient):
    return weight - learning_rate * gradient

gd =gradient_descent(x_train, h_pred, gradient)
print("gd = ", gd)
weights = update_weight_loss(weights, 0.1, gradient_descent(x_train, h_pred, y_actual))
print(" w =", weights)



The weights are updated by substracting the derivative (gradient descent) times the learning rate,